In [ ]:
import os

import torch
import wandb
from tqdm import tqdm
import seqdata as sd

import eugene as eu
import eugene.train
import eugene.models
from eugene import settings

In [ ]:
settings.dataset_dir = "/cellar/users/aklie/data/eugene/revision/kopp21"
settings.output_dir = "/cellar/users/dlaub/projects/ML4GLand/EUGENe_paper/output/kopp21"
settings.logging_dir = "/cellar/users/dlaub/projects/ML4GLand/EUGENe_paper/logs/kopp21"
settings.config_dir = "/cellar/users/dlaub/projects/ML4GLand/EUGENe_paper/configs/kopp21"
wandb.login()

In [ ]:
from pytorch_lightning import seed_everything
def prep_new_model(
    seed,
    conv_dropout = 0,
    dense_dropout = 0,
    batchnorm = True
):
    # Set a seed
    seed_everything(seed)

    model = models.zoo.DeepBind(
        input_len=41, # Length of padded sequences
        output_dim=1, # Number of multitask outputs
        conv_kwargs=dict(input_channels=4, conv_channels=[16], conv_kernels=[16], dropout_rates=conv_dropout, batchnorm=batchnorm),
        dense_kwargs=dict(hidden_dims=[32], dropout_rates=dense_dropout, batchnorm=batchnorm),
    )
    
    # Initialize the model prior to conv filter initialization
    models.init_weights(model)

    module = models.SequenceModule(
        arch=model,
        task="regression",
        loss_fxn="mse",
        optimizer="adam",
        optimizer_lr=0.0005,
        scheduler_kwargs=dict(patience=2)
    )

    # Return the model
    return module

In [ ]:
ray13 = prep_new_model(seed=13, conv_dropout=0.5, dense_dropout=0.5, batchnorm=True)